In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [33]:
# root = "/kaggle/input/jpx-tokyo-stock-exchange-prediction/"

# stock_prices = pd.read_csv(os.path.join(dirname, "stock_prices.csv"))

train = pd.read_csv("train.csv")



In [34]:
train.matched_size


# train["test_ratio"] = train.imbalance_buy_sell_flag*train.matched_size/train.imbalance_size

0          13380276.64
1           1642214.25
2           1819368.03
3          18389745.62
4          17860614.95
              ...     
5237975    28280361.74
5237976     9187699.11
5237977    12725436.10
5237978    94773271.05
5237979    24073677.32
Name: matched_size, Length: 5237980, dtype: float64

In [35]:

def make_data(df, train=0):
	data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

	params = ["imbalance_buy_sell", "reference_price", "matched_size", "wap_logreturns"]

	data["wap_logreturns"] = np.log(data.wap/data.wap.shift(1))

	data["matched_ratio"] = data.imbalance_buy_sell_flag*data.matched_size/data.imbalance_size

	data = data.dropna(subset=params)

	X = np.array([data.wap_logreturns, data.matched_ratio]).reshape(-1,2)

	if train == 0:
		y = np.array(data.target).reshape(-1,1)
		return X, y

	return X

# def make_data(df, train=0):
# 	data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

# 	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

# 	params = ["imbalance_buy_sell", "reference_price", "matched_size"]

# 	data = data.dropna(subset=params)
# 	X = np.array([data.wap, data.imbalance_buy_sell]).reshape(-1,2)

# 	if train == 0:
# 		y = np.array(data.target).reshape(-1,1)
# 		return X, y

# 	return X


In [36]:


X, y = make_data(train)

X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [37]:
model = LinearRegression()
model.fit(X_train,y_train)


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
model.coef_

predict_target = model.predict(X)

In [ ]:
# train[train.seconds_in_bucket > 10].head()
# train.loc[train.stock_id==1,['target', 'time_id', 'stock_id', 'seconds_in_bucket']]
train[train.stock_id==1]['seconds_in_bucket'].unique()

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540])

In [ ]:
pd.DataFrame(predict_target).describe()

,0
count,5.237539e+06
mean,-4.830525e-02
std,3.819503e-03
min,-3.495372e-01
25%,-4.833497e-02
50%,-4.833497e-02
75%,-4.833497e-02
max,6.818348e-01


In [ ]:

stats.describe(predict_target)



DescribeResult(nobs=5237539, minmax=(array([-0.34953717]), array([0.6818348])), mean=array([-0.04830525]), variance=array([1.4588603e-05]), skewness=array([26.89427685]), kurtosis=array([4535.75267927]))

In [ ]:
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()


In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))

    X = make_data(test, train=1)
    sample_prediction['target'] = model.predict(X)
    sample_prediction['target'] = 0
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                  0      3753451.43   
1         1      478                  0       985977.11   
2         2      478                  0       599128.74   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         0.999875   11548975.43        NaN   
1                       -1         1.000245    3850033.97        NaN   
2                        1         1.000584    4359198.25        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap   row_id  
0         NaN   0.999875  22940.00   1.000050   9177.60  1.0  478_0_0  
1         NaN   0.999940   1967.90   1.000601  19692.00  1.0  478_0_1  
2         NaN   0.999918   4488.22   1.000636  34955.12  1.0  478_0_2  
   stock_id  date_id  seconds_in_bucket  revealed_target  rev

In [ ]:
# for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
#     prices.head()
#     x_test = featuring_2(prices)
#     y_pred = model.predict(x_test)
#     sample_prediction['Target'] = y_pred
#     sample_prediction = sample_prediction.sort_values(by = "Target", ascending = False)
#     sample_prediction['Rank'] = np.arange(len(sample_prediction.index))
#     sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending = True)
#     sample_prediction.drop(["Target"], axis = 1)
#     submission = sample_prediction[["Date", "SecuritiesCode", "Rank"]]
#     env.predict(submission)

In [39]:
print(sample_prediction)

counter

NameError: name 'sample_prediction' is not defined

In [38]:
!head submission.csv
sub = pd.read_csv("submission.csv")


head: submission.csv: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'

In [29]:
sub[sub.target == 'target']

# type(sub.targetpd.to_numeric(df['id'], errors='coerce').notnull())
sub.target = pd.to_numeric(sub.target, errors='coerce')
sub = sub[sub.target.notnull()]


In [31]:
sub.target.max()

0.0971120314874603

In [1]:
!ls

example_test_files    optiver2023			 train.csv
__init__.py	      public_timeseries_testing_util.py
kaggle_optiver.ipynb  submission.csv
